In [3]:
# 🔄 Auto-reload modules when their source code changes
%load_ext autoreload
%autoreload 2

# 📦 Set up src/ and utils/ paths (project root is detected via pyproject.toml)
%run ../../bootstrap.py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import sys
import os

from print_torch_info import print_torch_info
from config import print_config_info

print_torch_info()
print_config_info()

🔍 PyTorch Environment Info
----------------------------------------
PyTorch version:     2.7.1
Available device:    MPS (Apple Silicon)
📂 Configured Dataset Directories
--------------------------------------------------
Raw data dir:             /Users/kenneth/Public/projects/python/ai/llms/llm-zero-to-trained/datasets/raw
Processed data dir:       /Users/kenneth/Public/projects/python/ai/llms/llm-zero-to-trained/datasets/processed
Vocab dir:                /Users/kenneth/Public/projects/python/ai/llms/llm-zero-to-trained/datasets/vocab
